In [1]:
 # 양방향 LSTM 이용한 NER 

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np


# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents


# 학습용 말뭉치 데이터를 불러옴
corpus = read_file('./data/train.txt')

# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
# 단어와 BIO 태그만 이용해 학습용 데이터셋 생성
# 0번째 원본 문장에서 분리된 단어 토큰들이 sentences 리스트에 저장
# sentences 리스트에 저장된 단어 시퀀스에 해당하는 BIO 태그 정보들이 tags 리스트에 저장
# sentences 리스트와 tags 리스트 크기는 동일
# 단어 시퀀스 평균 길이값을 기준으로 시퀀스 패딩 크기 결정
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[2]))
        sentence.append(w[0])
        bio_tag.append(w[2])

    sentences.append(sentence)
    tags.append(bio_tag)

print("샘플 크기 : \n", len(sentences))
print("0번째 샘플 문장 시퀀스 : \n", sentences[0])
print("0번째 샘플 bio 태그 : \n", tags[0])
print("샘플 문장 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 문장 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))


# 토크나이저 정의
# 위에서 만든 단어 시퀀스와 태그 시퀀스를 사전으로 만들기 위해 토크나이저 정의하고 fit_on_texts() 함수 호출 
sent_tokenizer = preprocessing.text.Tokenizer(oov_token='OOV') # 첫 번째 인덱스에는 OOV 사용(out of vocabulary) - 단어 사전에 포함하지 않는 단어 
sent_tokenizer.fit_on_texts(sentences)  # fit_on_texts() 문자 데이터 전달 받아 리스트 형태로 변환
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower= False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)

# 단어 사전 및 태그 사전 크기
# 생성된 사전 리스트를 이용해 단어와 태그 사전의 크기 정의
vocab_size = len(sent_tokenizer.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기 :", tag_size)
print("단어 사전 크기 :", vocab_size)

# 학습용 단어 시퀀스 생성
# 사전 데이터를 시퀀스 번호 형태로 인코딩
x_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)
print(x_train[0])
print(y_train[0])

# index to word / index to NER 정의
index_to_word = sent_tokenizer.index_word # 시퀀스 인덱스를 단어로 변환하기 위해 사용
index_to_ner = tag_tokenizer.index_word # 시퀀스 인덱스를 NER로 변환하기 위해 사용
index_to_ner[0] = 'PAD'

# 시퀀스 패딩 처리
# 개체명 인식 모델의 입출력 벡터 크기를 동일하게 맞추기 위해 시퀀스 패딩 작업
# 벡터 크기를 위-위에서 계산한 단어 시퀀스 평균 길이보다 넉넉하게 40으로 정의
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

# 학습 데이터와 테스트 데이터를 8:2 비율로 분리
# sklearn.model_selection 모듈 train_test_split() 함수 이용해 학습용, 테스트용 데이터셋을 8:2 비율로 분리 
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=.2, random_state=0)

# 출력 데이터를 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)

# 모델 정의(Bi-LSTM)
# 개체 인식 모델 순차 모델 방식 구현
# tag_size 만큼 출력 뉴런에서 제일 확률 높은 출력값 1개 선택하는 문제라 모델 출력 계층 활성화 함수로 softmax 사용
# 손실함수 categorical_crossentropy 사용
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))  # 단어 임베딩 
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))  # 양방향 LSTM 
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))  # NER 태그분류 
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)
print("평가 결과 : ", model.evaluate(x_test, y_test)[1])  # 학습 평가 결과 확인


# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):
    result = []
    for sequence in sequences:
        temp = []
        for pred in sequence:
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))
        result.append(temp)
    return result


# 테스트 데이터셋의 NER 예측
# F1스코어 계산하기 위해 모델 predict() 함수 통해 테스트용 데이터셋 결과 예측 
# 해당 함수 결과로 예측된 NER 태그 정보가 담긴 넘파이 배열이 반환
y_predicted = model.predict(x_test) # (711, 40) => model => (711, 40, 8)
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test) # 실제 NER

# F1 스코어 계산을 위해 사용
# seqeval.metrics 모듈의 classification_report() 함수 통해 NER 태그별로 계산된 정밀도 재현율 F!스코어 출력
# f1스코어만 불러올 수도 있음 - f1_score() 함수 
from seqeval.metrics import f1_score, classification_report
print(classification_report(test_tags, pred_tags))
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))


# 새로운 유형의 문장 NER 예측
word_to_index = sent_tokenizer.word_index
new_sentence = '삼성전자 출시 스마트폰 오늘 애플 아이폰에 도전장 내밀다.'.split()
new_x = []
for w in new_sentence:
    try:
        new_x.append(word_to_index.get(w, 1))
    except KeyError:
        # 모르는 단어의 경우 OOV
        new_x.append(word_to_index['OOV'])

print("새로운 유형의 시퀀스 : ", new_x)
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding="post", value=0, maxlen=max_len)

# NER 예측
p = model.predict(np.array([new_padded_seqs[0]]))
p = np.argmax(p, axis=-1) # 예측된 NER 인덱스값 추출
print("{:10} {:5}".format("단어", "예측된 NER"))
print("-" * 50)

for w, pred in zip(new_sentence, p[0]):
    print("{:10} {:5}".format(w, index_to_ner[pred]))

샘플 크기 : 
 3555
0번째 샘플 문장 시퀀스 : 
 ['1', '1', '2', '2', '2', '3', '3', '3', '4', '4', '5', '6', '7', '8', '9', '9', '10', '10', '10', '11']
0번째 샘플 bio 태그 : 
 ['NNG', 'SP', 'SL', 'NNG', 'NNG', 'SL', 'NNG', 'JKB', 'VV', 'ETM', 'NNP', 'MAJ', 'NNG', 'NNG', 'NNG', 'JKS', 'NNG', 'VV', 'EC', 'SF']
샘플 문장 시퀀스 최대 길이 : 168
샘플 문장 시퀀스 평균 길이 : 34.03909985935302
BIO 태그 사전 크기 : 46
단어 사전 크기 : 81
[6, 6, 4, 4, 4, 3, 3, 3, 2, 2, 5, 7, 8, 9, 10, 10, 11, 11, 11, 12]
[1, 16, 19, 1, 1, 19, 1, 6, 3, 7, 4, 33, 1, 1, 1, 15, 1, 3, 2, 11]
학습 샘플 시퀀스 형상 :  (2844, 40)
학습 샘플 레이블 형상 :  (2844, 40, 46)
테스트 샘플 시퀀스 형상 :  (711, 40)
테스트 샘플 레이블 형상 :  (711, 40, 46)
Epoch 1/10
23/23 [==============================] - 59s 1s/step - loss: 2.9562 - accuracy: 0.2740
Epoch 2/10
23/23 [==============================] - 28s 1s/step - loss: 2.6428 - accuracy: 0.3279
Epoch 3/10
23/23 [==============================] - 23s 979ms/step - loss: 2.5916 - accuracy: 0.3491
Epoch 4/10
23/23 [==============================] - 23s 976ms/step - loss

c:\Users\Playdata\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: VV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\Playdata\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ETM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\Playdata\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: SN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\Playdata\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: NNB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\Playdata\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: NNP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\Playdata\Anaconda3\lib\site-packages\seqeval\met

              precision    recall  f1-score   support

           A       0.00      0.00      0.00       158
          AG       0.00      0.00      0.00       223
          AJ       0.00      0.00      0.00        44
          BK       0.00      0.00      0.00         1
           C       0.62      0.16      0.26      1303
          CN       0.00      0.00      0.00        10
          CP       0.00      0.00      0.00       231
           F       0.10      0.89      0.18      1097
           H       0.00      0.00      0.00         9
          KB       0.14      0.25      0.18       895
          KC       0.00      0.00      0.00         4
          KG       0.00      0.00      0.00       313
          KO       0.28      0.07      0.11       718
          KQ       0.00      0.00      0.00         8
          KS       0.50      0.02      0.05       376
          KV       0.00      0.00      0.00         1
           L       0.00      0.00      0.00       299
           M       0.00    